In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\4\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\4\test'
checkpoint_path = r'C:\Users\24306\Desktop\EfficientNetV2L\4\EfficientNetV2L.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[40.0, 32.0, 37.0, 45.0, 80.0, 40.0, 47.0, 80.0, 13.0, 66.0, 7.0, 13.0, 37.0, 100.0, 80.0, 95.0, 10.0, 10.0, 96.0, 30.0, 52.0, 32.0, 80.0, 30.0, 84.0, 66.0, 93.0, 10.0, 66.0, 100.0, 95.0, 80.0, 37.0, 80.0, 95.0, 52.0, 66.0, 52.0, 86.0, 84.0, 20.0, 84.0, 52.0, 13.0, 100.0, 16.0, 10.0, 96.0, 80.0, 100.0, 47.0, 40.0, 23.0, 95.0, 30.0, 32.0, 37.0, 13.0, 45.0, 47.0, 35.0, 84.0, 40.0, 32.0, 95.0, 45.0, 16.0, 80.0, 80.0, 95.0, 45.0, 4.0, 10.0, 55.0, 7.0, 95.0, 37.0, 16.0, 37.0, 10.0, 66.0, 86.0, 100.0, 96.0, 37.0, 7.0, 93.0, 13.0, 32.0, 93.0, 55.0, 55.0, 80.0, 66.0, 52.0, 37.0, 93.0, 45.0, 45.0, 80.0, 55.0, 7.0, 84.0, 20.0, 35.0, 95.0, 13.0, 47.0, 45.0, 37.0, 100.0, 66.0, 55.0, 16.0, 93.0, 45.0, 84.0, 37.0, 23.0, 100.0, 47.0, 45.0, 37.0, 37.0, 93.0, 45.0, 52.0, 93.0, 100.0, 13.0, 100.0, 40.0, 13.0, 45.0, 93.0, 13.0, 13.0, 7.0, 80.0, 47.0, 95.0, 40.0, 4.0, 13.0, 40.0, 80.0, 66.0, 96.0, 66.0, 66.0, 30.0, 7.0, 100.0, 47.0, 35.0, 16.0, 66.0, 45.0, 95.0, 84.0, 86.0, 93.0, 47.0, 37.0, 80.0, 45.0, 4

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
EfficientNetV2L = tf.keras.applications.efficientnet_v2.EfficientNetV2L(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
EfficientNetV2L.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = EfficientNetV2L(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-l (Functiona  (None, 7, 7, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1281      
                                                                 
Total params: 117,748,129
Trainable params: 117,235,553
Non-trainable params: 512,576
_________________________________________________________________


In [ ]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('EfficientNetV2L_loss_4.txt',history.history['loss'])
#要改
np.savetxt('EfficientNetV2L_valloss_4.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Mon May 15 15:42:12 2023
Epoch 1/300
592/592 [==============================] - ETA: 0s - loss: 1076.4916

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\4\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\4\EfficientNetV2L.ckpt\assets


592/592 [==============================] - 617s 967ms/step - loss: 1076.4916 - val_loss: 556.0870
Epoch 2/300
592/592 [==============================] - ETA: 0s - loss: 331.0740

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\4\EfficientNetV2L.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\4\EfficientNetV2L.ckpt\assets


592/592 [==============================] - 1259s 2s/step - loss: 0.4644 - val_loss: 0.0789
Epoch 124/300
592/592 [==============================] - 1027s 2s/step - loss: 0.3495 - val_loss: 0.1141
Epoch 125/300
592/592 [==============================] - 1044s 2s/step - loss: 0.4784 - val_loss: 0.3067
Epoch 126/300
592/592 [==============================] - 1052s 2s/step - loss: 0.6021 - val_loss: 0.2422
Epoch 127/300
592/592 [==============================] - 1051s 2s/step - loss: 0.4138 - val_loss: 0.2008
Epoch 128/300
592/592 [==============================] - ETA: 0s - loss: 0.5474

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\4\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\4\EfficientNetV2L.ckpt\assets


592/592 [==============================] - 1298s 2s/step - loss: 0.5474 - val_loss: 0.0714
Epoch 129/300
592/592 [==============================] - 973s 2s/step - loss: 0.3429 - val_loss: 0.0902
Epoch 130/300
592/592 [==============================] - 976s 2s/step - loss: 0.2630 - val_loss: 0.1043
Epoch 131/300
592/592 [==============================] - 974s 2s/step - loss: 0.2902 - val_loss: 0.1197
Epoch 132/300
592/592 [==============================] - 970s 2s/step - loss: 0.4235 - val_loss: 0.1549
Epoch 133/300
592/592 [==============================] - 968s 2s/step - loss: 0.2514 - val_loss: 0.1217
Epoch 134/300
592/592 [==============================] - 966s 2s/step - loss: 0.3030 - val_loss: 0.3183
Epoch 135/300
592/592 [==============================] - 964s 2s/step - loss: 0.3523 - val_loss: 0.1188
Epoch 136/300
592/592 [==============================] - 964s 2s/step - loss: 0.3751 - val_loss: 0.3599
Epoch 137/300
592/592 [==============================] - 964s 2s/step - loss: